In [30]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import lightgbm as lgb
import bisect
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold

%matplotlib inline
plt.style.use(['dark_background'])
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',100)

In [31]:
# 데이터 불러오기
train = pd.read_csv('train.csv').drop(columns=['SAMPLE_ID'])
test = pd.read_csv('test.csv').drop(columns=['SAMPLE_ID'])

In [32]:
# 1. year, month, day, hour, minute, weekday 컬럼 그대로 생성
# datetime 컬럼 처리
train['ATA'] = pd.to_datetime(train['ATA'])
test['ATA'] = pd.to_datetime(test['ATA'])

# datetime을 여러 파생 변수로 변환
for df in [train, test]:
    df['year'] = df['ATA'].dt.year
    df['month'] = df['ATA'].dt.month
    df['day'] = df['ATA'].dt.day
    df['hour'] = df['ATA'].dt.hour
    df['minute'] = df['ATA'].dt.minute
    df['weekday'] = df['ATA'].dt.weekday


# 2. datetime 컬럼 제거
train.drop(columns='ATA', inplace=True)
test.drop(columns='ATA', inplace=True)

'''
# 3. Categorical 컬럼 인코딩
categorical_features = ['ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'ID', 'SHIPMANAGER', 'FLAG']
encoders = {}

for feature in tqdm(categorical_features, desc="Encoding features"):
    le = LabelEncoder()
    train[feature] = le.fit_transform(train[feature].astype(str))
    le_classes_set = set(le.classes_)
    test[feature] = test[feature].map(lambda s: '-1' if s not in le_classes_set else s)
    le_classes = le.classes_.tolist()
    bisect.insort_left(le_classes, '-1')
    le.classes_ = np.array(le_classes)
    test[feature] = le.transform(test[feature].astype(str))
    encoders[feature] = le

'''


'\n# 3. Categorical 컬럼 인코딩\ncategorical_features = [\'ARI_CO\', \'ARI_PO\', \'SHIP_TYPE_CATEGORY\', \'ID\', \'SHIPMANAGER\', \'FLAG\']\nencoders = {}\n\nfor feature in tqdm(categorical_features, desc="Encoding features"):\n    le = LabelEncoder()\n    train[feature] = le.fit_transform(train[feature].astype(str))\n    le_classes_set = set(le.classes_)\n    test[feature] = test[feature].map(lambda s: \'-1\' if s not in le_classes_set else s)\n    le_classes = le.classes_.tolist()\n    bisect.insort_left(le_classes, \'-1\')\n    le.classes_ = np.array(le_classes)\n    test[feature] = le.transform(test[feature].astype(str))\n    encoders[feature] = le\n\n'

In [33]:
# 4. DIST가 0이면, CI_HOUR 모두 0으로 변경
train.loc[train['DIST'] == 0, 'CI_HOUR'] = 0

print(train)

       ARI_CO ARI_PO SHIP_TYPE_CATEGORY       DIST       ID  BREADTH  BUILT  \
0          SG   GIW5          Container  30.881018  Z618338     30.0     24   
1          IN   UJM2               Bulk   0.000000  X886125     30.0     13   
2          CN   EUC8          Container   0.000000  T674582     50.0     12   
3          JP   ZAG4          Container   0.000000  Y847238     20.0     18   
4          SG   GIW5          Container  27.037650  A872328     50.0     10   
...       ...    ...                ...        ...      ...      ...    ...   
391934     JP   QYY1          Container   0.000000  Y375615     20.0     27   
391935     SG   GIW5               Bulk   5.884603  K635567     10.0     12   
391936     US   QGN3               Bulk  70.660241  J284147     30.0      8   
391937     TW   JWI3          Container   9.448179  J644215     30.0     29   
391938     TW   JWI3          Container   0.000000  D123358     30.0     15   

        DEADWEIGHT  DEPTH  DRAUGHT      GT  LENGTH 

In [34]:
# 5. 결측치가 존재하는 데이터 1개 삭제
train = train.dropna(subset=['LENGTH']).reset_index(drop=True)

In [35]:

# 6-1. 기상 관련 결측치 많은 컬럼 제거
train.drop(columns = 'U_WIND', inplace = True)
train.drop(columns = 'V_WIND', inplace = True)
train.drop(columns = 'AIR_TEMPERATURE', inplace = True)
train.drop(columns = 'BN', inplace = True)

test.drop(columns = 'U_WIND', inplace = True)
test.drop(columns = 'V_WIND', inplace = True)
test.drop(columns = 'AIR_TEMPERATURE', inplace = True)
test.drop(columns = 'BN', inplace = True)


In [36]:
# 6-2. 기상 관련 결측치 KNN을 통한 대체
'''
from sklearn.impute import KNNImputer

### 결측치 KNNImputer를 통해 대체

# K-NN 대체기
knn_imputer = KNNImputer(n_neighbors=10)

# K-NN 대체를 적용할 열을 지정
columns_to_impute = ['U_WIND', 'V_WIND', 'AIR_TEMPERATURE', 'BN']

# train 데이터프레임에서 결측치를 K-NN으로 대체
filtered_train[columns_to_impute] = knn_imputer.fit_transform(filtered_train[columns_to_impute])

filtered_train = pd.DataFrame(filtered_train, columns=filtered_train.columns)
#filtered_train.to_csv('train_imputed.csv', index = False)

'''

"\nfrom sklearn.impute import KNNImputer\n\n### 결측치 KNNImputer를 통해 대체\n\n# K-NN 대체기\nknn_imputer = KNNImputer(n_neighbors=10)\n\n# K-NN 대체를 적용할 열을 지정\ncolumns_to_impute = ['U_WIND', 'V_WIND', 'AIR_TEMPERATURE', 'BN']\n\n# train 데이터프레임에서 결측치를 K-NN으로 대체\nfiltered_train[columns_to_impute] = knn_imputer.fit_transform(filtered_train[columns_to_impute])\n\nfiltered_train = pd.DataFrame(filtered_train, columns=filtered_train.columns)\n#filtered_train.to_csv('train_imputed.csv', index = False)\n\n"

In [37]:
# 데이터프레임 전체 결측치 파악
total_missing = train.isna().sum()
print(total_missing)

ARI_CO                0
ARI_PO                0
SHIP_TYPE_CATEGORY    0
DIST                  0
ID                    0
BREADTH               0
BUILT                 0
DEADWEIGHT            0
DEPTH                 0
DRAUGHT               0
GT                    0
LENGTH                0
SHIPMANAGER           0
FLAG                  0
ATA_LT                0
PORT_SIZE             0
CI_HOUR               0
year                  0
month                 0
day                   0
hour                  0
minute                0
weekday               0
dtype: int64


In [38]:
# 7. 다중공선성 문제 해결을 위해 Length 컬럼 이외 삭제 (BREADTH, DEADWEIGHT, DEPTH, DRAUGHT, GT)

for df in [train, test]:
    df.drop(columns = 'BREADTH', inplace = True)
    df.drop(columns = 'DEADWEIGHT', inplace = True)
    df.drop(columns = 'DEPTH', inplace = True)
    df.drop(columns = 'DRAUGHT', inplace = True)
    df.drop(columns = 'GT', inplace = True)

In [39]:
#  8. CI_HOUR 컬럼에서 표준편차 + 2 * 표준편차 이상이 되는 부분 이상치라고 판단 후, 삭제

# 'CI_HOUR' 열의 평균과 표준 편차 계산
mean = train['CI_HOUR'].mean()
std = train['CI_HOUR'].std()

# 이상치 경계 설정 (예: 평균에서 2배 표준 편차를 벗어나는 값)
lower_bound = mean - 2 * std
upper_bound = mean + 2 * std

# 이상치를 제거하고 정상 범위의 데이터만 남김
#filtered_train = train[(train['CI_HOUR'] >= lower_bound) & (train['CI_HOUR'] <= upper_bound)]
train = train[(train['CI_HOUR'] >= lower_bound) & (train['CI_HOUR'] <= upper_bound)].reset_index(drop=True)

In [40]:
# 10월 20일 코드추가
# 주말여부 추가
# 시간(hour)는 cyclical encoding하여 변수 추가(sin time & cos time)

In [41]:
# 주말여부 추가
train['holiday'] = train.apply(lambda x : 0 if x['day']<5 else 1, axis = 1)
test['holiday'] = test.apply(lambda x : 0 if x['day']<5 else 1, axis = 1)

In [42]:
# 시간(hour)/month/minute/day/weekday는 cyclical encoding하여 변수 추가(sin time & cos time) 후 삭제
# day는 30,31 애매해서 제외

for df in [train, test]:
  # 시간
  df['sin_time_hour'] = np.sin(2*np.pi*train.hour/24)
  df['cos_time_hour'] = np.cos(2*np.pi*train.hour/24)
  # 월
  df['sin_time_month'] = np.sin(2*np.pi*train.month/12)
  df['cos_time_month'] = np.cos(2*np.pi*train.month/12)
  # weekday
  df['sin_time_weekday'] = np.sin(2*np.pi*train.weekday/7)
  df['cos_time_weekday'] = np.cos(2*np.pi*train.weekday/7)
  # minute
  df['sin_time_minute'] = np.sin(2*np.pi*train.minute/60)
  df['cos_time_minute'] = np.cos(2*np.pi*train.minute/60)

In [43]:
train

,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ID,BUILT,LENGTH,SHIPMANAGER,FLAG,ATA_LT,PORT_SIZE,CI_HOUR,year,month,day,hour,minute,weekday,holiday,sin_time_hour,cos_time_hour,sin_time_month,cos_time_month,sin_time_weekday,cos_time_weekday,sin_time_minute,cos_time_minute
0,SG,GIW5,Container,30.881018,Z618338,24,180.0,CQSB78,Panama,5,0.002615,3.450000,2018,12,17,21,29,0,1,-0.707107,7.071068e-01,-2.449294e-16,1.000000e+00,0.000000,1.000000,1.045285e-01,-9.945219e-01
1,IN,UJM2,Bulk,0.000000,X886125,13,180.0,SPNO34,Marshall Islands,12,0.000217,0.000000,2014,9,23,6,59,1,1,1.000000,6.123234e-17,-1.000000e+00,-1.836970e-16,0.781831,0.623490,-1.045285e-01,9.945219e-01
2,CN,EUC8,Container,0.000000,T674582,12,370.0,FNPK22,Malta,6,0.001614,0.000000,2015,2,3,22,0,1,0,-0.500000,8.660254e-01,8.660254e-01,5.000000e-01,0.781831,0.623490,0.000000e+00,1.000000e+00
3,JP,ZAG4,Container,0.000000,Y847238,18,120.0,PBZV77,Bahamas,13,0.000356,0.000000,2020,1,17,4,2,4,1,0.866025,5.000000e-01,5.000000e-01,8.660254e-01,-0.433884,-0.900969,2.079117e-01,9.781476e-01
4,SG,GIW5,Container,27.037650,A872328,10,300.0,GUCE76,Liberia,15,0.002615,253.554444,2020,1,26,7,51,6,1,0.965926,-2.588190e-01,5.000000e-01,8.660254e-01,-0.781831,0.623490,-8.090170e-01,5.877853e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379812,JP,QYY1,Container,0.000000,Y375615,27,110.0,KEJZ24,"China, People's Republic Of",14,0.000552,0.000000,2017,6,6,5,2,1,1,0.965926,2.588190e-01,1.224647e-16,-1.000000e+00,0.781831,0.623490,2.079117e-01,9.781476e-01
379813,SG,GIW5,Bulk,5.884603,K635567,12,80.0,JLTM64,Vietnam,8,0.002615,144.061389,2019,10,16,0,36,2,1,0.000000,1.000000e+00,-8.660254e-01,5.000000e-01,0.974928,-0.222521,-5.877853e-01,-8.090170e-01
379814,US,QGN3,Bulk,70.660241,J284147,8,200.0,YERJ68,Singapore,18,0.000155,41.482222,2021,3,23,22,35,1,1,-0.500000,8.660254e-01,1.000000e+00,6.123234e-17,0.781831,0.623490,-5.000000e-01,-8.660254e-01
379815,TW,JWI3,Container,9.448179,J644215,29,170.0,HCZK58,Comoros,15,0.000990,7.485278,2015,1,8,7,15,3,1,0.965926,-2.588190e-01,5.000000e-01,8.660254e-01,0.433884,-0.900969,1.000000e+00,2.832769e-16


In [44]:
test

,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ID,BUILT,LENGTH,SHIPMANAGER,FLAG,ATA_LT,PORT_SIZE,year,month,day,hour,minute,weekday,holiday,sin_time_hour,cos_time_hour,sin_time_month,cos_time_month,sin_time_weekday,cos_time_weekday,sin_time_minute,cos_time_minute
0,SG,GIW5,Container,1.826589,K322654,18,340.0,KQMD74,Panama,19,0.002615,2020,6,18,11,58,3,1,-0.707107,7.071068e-01,-2.449294e-16,1.000000e+00,0.000000,1.000000,1.045285e-01,-0.994522
1,CN,WEY7,Cargo,25.399386,E376681,13,80.0,LBYS27,Belize,6,0.001028,2021,5,26,22,20,2,1,1.000000,6.123234e-17,-1.000000e+00,-1.836970e-16,0.781831,0.623490,-1.045285e-01,0.994522
2,CN,NGG6,Container,111.079467,F811111,26,140.0,EKFV15,Singapore,8,0.001743,2019,12,16,0,9,0,1,-0.500000,8.660254e-01,8.660254e-01,5.000000e-01,0.781831,0.623490,0.000000e+00,1.000000
3,CA,FFM2,Bulk,9.175258,A737561,9,190.0,MASW32,Panama,2,0.000182,2015,11,16,5,30,0,1,0.866025,5.000000e-01,5.000000e-01,8.660254e-01,-0.433884,-0.900969,2.079117e-01,0.978148
4,JP,QYY1,Container,0.000000,A827175,19,220.0,SIEL54,Liberia,10,0.000552,2018,10,24,1,11,2,1,0.965926,-2.588190e-01,5.000000e-01,8.660254e-01,-0.781831,0.623490,-8.090170e-01,0.587785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220486,BR,TMW2,Bulk,21.712733,E731427,15,330.0,UWHE18,Japan,13,0.000080,2017,4,4,15,53,1,0,-0.500000,8.660254e-01,5.000000e-01,-8.660254e-01,-0.781831,0.623490,-2.079117e-01,0.978148
220487,CA,GRQ5,Bulk,4.870490,Q434522,9,220.0,BBXV46,Liberia,13,0.000039,2016,8,22,16,22,0,1,0.866025,-5.000000e-01,-1.000000e+00,-1.836970e-16,0.781831,0.623490,-9.510565e-01,0.309017
220488,JP,VYJ1,Tanker,17.068286,U148516,14,60.0,DYUA26,Japan,23,0.000264,2022,7,10,14,53,6,1,0.965926,2.588190e-01,-1.000000e+00,-1.836970e-16,0.781831,0.623490,5.665539e-16,-1.000000
220489,CN,QQW1,Cargo,0.000000,C235264,27,80.0,VKVS56,Belize,22,0.000595,2020,12,28,14,38,0,1,0.965926,2.588190e-01,1.000000e+00,6.123234e-17,0.433884,-0.900969,0.000000e+00,1.000000


In [47]:
train.to_csv('./train_preprocessed_10.20.csv')
test.to_csv('./test_preprocessed_10.20.csv')